## Notation
**Causal identification**

- Observed covariates/features: $X$

- Potential outcomes: $Y(0)$ and $Y(1)$

- Treatment: $T$

- Unobservable Individual Treatment Effect: $\tau_i = Y_i(1) - Y_i(0)$

- Average Treatment Effect: $ATE =\mathbb{E}[Y_i(1)-Y_i(0)]= \mathbb{E}[{\tau_i}]$

- Conditional Average Treatment Effect: $CATE(x) =\mathbb{E}[Y_i(1)-Y_i(0)|X=x]$


**Deep learning estimation**

- Predicted outcomes: $\hat{Y}(0)$ and $\hat{Y}(1)$

- Outcome modeling functions: $\hat{Y}(T)=h(X,T)$ 

- Representation functions: $\Phi(X)$

- Propensity score function:
$\pi(X,T)=P(T|X)$ </br>*where $\pi(X,1)=P(T=1|X)$ and $\pi(X,0)=1-\pi(X,1)$* 

- Loss functions: $\mathcal{L}(true,predicted)$, with the mean squared error abbreviated $MSE$ and binary cross-entropy as $BCE$

- Estimated CATE<sup>*</sup>: $\hat{CATE_i} = \hat{\tau}_i = \hat{Y_i}(1)-\hat{Y_i}(0) = h(X,1)-h(X,0)$

- Estimated ATE: $\hat{ATE}=\frac{1}{n}\sum_{i=1}^n\hat{CATE_i}$

- Nearest-neighbor PEHE:
$$PEHE_{nn}=\frac{1}{N}\sum_{i=1}^N{(\underbrace{(1−2t_i)(y_i(t_i)−y_i^{nn}(1-t_i)}_{CATE_{nn}}−\underbrace{(h(\Phi(x),1)−h(\Phi(x),0)))}_{\hat{CATE}}}^2$$ for nearest neighbor $j$ of each unit $i$ in representation space such that $t_j\neq t_i$:
  $$y_i^{nn}(1-t_i) = \min_{j\in (1-T)}||\Phi(x_i|t_i)-\Phi(x_j|1-t_i)||_2$$

\* We define $\hat{\tau}_i = \hat{CATE_i}$ because the we lack the covariates to estimate the ITE.  

## What are we doing here?

These model are designed to estimate the  average treatment effect (ATE) and the conditional average treatment effect(CATE) under a selection on observables identification strategy. The ATE is defined as:
 
$$ATE =\mathbb{E}[Y_i(1)-Y_i(0)]= \mathbb{E}[{\tau_i}]$$
 
where $Y_i(1)$ and $Y_i(0)$ are the potential outcomes had unit $i$ received or not received the treatment, respectively. The CATE is defined as,
 
$$CATE(x) =\mathbb{E}[Y_i(1)-Y_i(0)|X=x]$$

where $X$ is the set of selected, observable covariates, and $x \in X$.

## Training metrics for causal inference

Although our ultimate goal is to estimate the $\hat{CATE}$, the loss function in TARNet only minimizes the factual error to estimate $\hat{Y}$. This is a reflection of the fundamental problem of causal inference: we only observe one potential outcome for each unit.

Within this literature, it is common practice to evaluate model performance on simulations using the Precision Estimation of Heterogeneous  Effects (PEHE) from [Hill, 2011](https://www.tandfonline.com/doi/abs/10.1198/jcgs.2010.08162?casa_token=b8-rfzagECIAAAAA:QeP7C4lKN6nZ7MkDjJHFrEberXopD9M5qPBMeBqbk84mI_8qGxj01ctgt4jdZtORpu9aZvpVRe07PA). PEHE measures the error in estimates of the $CATE$:

$$PEHE=\frac{1}{N}\sum_{i=1}^N(CATE_i-\hat{CATE_i})^2$$

In order to select hyperparameters in real data, [Johansson et al., 2020](https://arxiv.org/pdf/2001.07426.pdf) propose to use a matching variant of $PEHE$ with the nearest Euclidean neighbor of each unit $i$ from the other treatment assignment group $y_i^{nn}$ as a counterfactual. If we identify the nearest neighbor $j$ of each unit $i$ in representation space such that $t_j\neq t_i$ as

  $$y_i^{nn}(1-t_i) = \min_{j\in (1-T)}||\Phi(x_i|t_i)-\Phi(x_j|1-t_i)||_2$$
 then,
$$PEHE_{nn}=\frac{1}{N}\sum_{i=1}^N{(\underbrace{(1−2t_i)(y_i(t_i)−y_i^{nn}(1-t_i)}_{CATE_{nn}}−\underbrace{(h(\Phi(x),1)−h(\Phi(x),0)))}_{\hat{CATE}}}^2$$
If we take the square root of the $PEHE_{nn}$ then we get an approximation of the unit-level error.

I think the intuition behind $\sqrt{PEHE_{nn}}$ is solid. If our representation function $\Phi$ is truly learning to balance the treated and control distributions, $CATE_{nn}$ should coarsely measure it.


# Representation learning as a balancing strategy 


A core concept in deep learning is the idea that artificial neural networks have the capacity to project a set of complex features $X$ into a useful vector space. When data are transformed into this space, we call the resulting tensor a **representation** ([Goodfellow, et al. 2016](https://www.deeplearningbook.org/contents/representation.html)) (you might also see the term "embedding"). For social scientists most comfortable with linear models, we can think about the parameters in each feed-forward layer of a deep neural network as capturing every possible interaction between the values produced by the previous layer. Tasking the network to minimize error on a relevant downstream task encourages it to adjust these interaction parameters to learn useful representations. We can also think about these representation layers as automatically extracting useful  latent covariates/features.

The key intuition in this literature is that we want to train neural networks to learn a representation function $\Phi(X)$ where the data are deconfounded/balanced in the representation space. In other words, the distributions of the representations $\Phi(X|T=0)$ and $\Phi(X|T=1)$ are similar.

<figure><img src=https://github.com/Gloriagao0624/Uplift/blob/main/WechatIMG77.jpeg?raw=true width="900"></figure>

Note that $\Phi$ must, in theory, be an invertible function for the  ignorability and overlap assumptions to hold. By invertible we mean that there is an inverse function such that $\Phi^{-1}(\Phi(X))=X$.
 

# Part 1: TARNet
To encourage balanced representations, [Shalit et al., 2017](http://proceedings.mlr.press/v70/shalit17a/shalit17a.pdf) propose a simple two-headed neural network called Treatment Agnostic Regression Network (TARNet). Each head models a separate outcome. One head learns the function $\hat{Y}(1)=h(\Phi(X),1)$, and the other head learns the function $\hat{Y}(0)=h(\Phi(X),0)$. Both heads backpropagate their gradients to shared representation layers that learn $\Phi(X)$. Again, the hope is that these representation layers will learn to balance the data because they are used to predict both outcomes.

<figure><img src=https://github.com/Gloriagao0624/Uplift/blob/main/WechatIMG60.jpeg?raw=true width="900">This architecture, originally introduced in <a href=http://proceedings.mlr.press/v70/shalit17a/shalit17a.pdf>Shalit et al., 2017</a>, is a T-learner with shared representation layers.</figcaption></figure>


Other than this architectural change, this has the same loss as the T-Learner:

$$\mathcal{L}(Y,h(\Phi(X),T))=MSE(Y,h(\Phi(X),T))=\frac{1}{n}\sum_{i=1}^n [h(\Phi(x_i),t_i)-y_i(t_i)]^2$$

 The complete objective for the network is to minimize the parameters of $h$ and $\Phi$ for all $n$ units in the training sample such that,

\begin{equation}
\min_{h,\Phi}\frac{1}{n}\sum_{i=1}^n \mathcal{L}(y_i(t_i),h(\Phi(x_i),t_i)) + \lambda \mathcal{R}(h)\end{equation}

where $\mathcal{R}(h)$ is a model complexity term (e.g., for $L_2$ regularization) and $\lambda$ is a hyperparameter chosen by the user.

## Loading the Data


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
from pathlib import Path
data_path = Path("")
df = pd.read_csv(data_path / "data.csv")
df.rename(columns={'in_ad_group': 'treatment_flg'}, inplace=True)
df.rename(columns={'return_label': 'target'}, inplace=True)
df = df.loc[(df['country'] =='eg') | (df['country'] =='iq') | (df['country'] =='sa')]
#df = df.loc[(df['sample'] =='random_ua') | (df['sample'] =='control_no_ua')]
df = df.drop('sample',axis=1)
df = df.drop('country',axis=1)
df = df.drop('level0',axis=1)
df = df.drop('uninstall_cnt',axis=1)
df = df.drop('now_uninstall',axis=1)
df.sort_values(by=['days_since_reg'],ascending=True)
df = df.drop_duplicates(['vopenid'])
df = df.set_index("vopenid")

In [3]:
np.random.seed(seed=2) 
typicalNDict = {1: 10000, 0: 20000}

def typicalsamling(group, typicalNDict):
    name = group.name
    n = typicalNDict[name]
    return group.sample(n=n)

df = df.groupby('target').apply(typicalsamling, typicalNDict)
print(df.shape)

(30000, 31)


In [4]:
y=df['target'][:,].astype('float32') #most GPUs only compute 32-bit floats
t=df['treatment_flg'][:,].astype('float32')
x = df.drop(columns=['target', 'treatment_flg']).astype('float32')
standard_scaler_data = StandardScaler().fit_transform(x)
df={'x':standard_scaler_data,'t':t,'y':y,'t':t}
df['t']=df['t'].values.reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension 
df['y']=df['y'].values.reshape(-1,1)
xt = np.concatenate([x, df['t']], 1)

## Adding our metrics to tensorflow

In [5]:
#!pip install -q tensorflow==2.8.0
import tensorflow as tf
import numpy as np
import datetime
%load_ext tensorboard 

In [6]:
from tensorflow.keras.callbacks import Callback

def pdist2sq(x,y):
    x2 = tf.reduce_sum(x ** 2, axis=-1, keepdims=True)
    y2 = tf.reduce_sum(y ** 2, axis=-1, keepdims=True)
    dist = x2 + tf.transpose(y2, (1, 0)) - 2. * x @ tf.transpose(y, (1, 0))
    return dist

'''
def pdist2sq(A, B):
    #helper for PEHEnn
    #calculates squared euclidean distance between rows of two matrices  
    #https://gist.github.com/mbsariyildiz/34cdc26afb630e8cae079048eef91865
    # squared norms of each row in A and B
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)    
    # na as a row and nb as a column vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])
    # return pairwise euclidean difference matrix
    D = tf.sqrt(tf.maximum(na - 2*tf.matmul(A, B, False, True) + nb, 0.0))
    return D
'''

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class Base_Metrics(Callback):
    def __init__(self,data, verbose=0):   
        super(Base_Metrics, self).__init__()
        self.data=data #feed the callback the full dataset
        self.verbose=verbose

        #needed for PEHEnn; Called in self.find_ynn
        self.data['o_idx']=tf.range(self.data['t'].shape[0])
        self.data['c_idx']=self.data['o_idx'][self.data['t'].squeeze()==0] #These are the indices of the control units
        self.data['t_idx']=self.data['o_idx'][self.data['t'].squeeze()==1] #These are the indices of the treated units
    
    def split_pred(self,concat_pred):
        #this helps us keep ptrack of things so we don't make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0].reshape(-1, 1)
        preds['y1_pred'] = concat_pred[:, 1].reshape(-1, 1)
        preds['phi'] = concat_pred[:, 2:]
        return preds

    def find_ynn(self, Phi):
        #helper for PEHEnn
        PhiC, PhiT =tf.dynamic_partition(Phi,tf.cast(tf.squeeze(self.data['t']),tf.int32),2) #separate control and treated reps
        dists=tf.sqrt(pdist2sq(PhiC,PhiT)) #calculate squared distance then sqrt to get euclidean
        yT_nn_idx=tf.gather(self.data['c_idx'],tf.argmin(dists,axis=0),1) #get c_idxs of smallest distances for treated units
        yC_nn_idx=tf.gather(self.data['t_idx'],tf.argmin(dists,axis=1),1) #get t_idxs of smallest distances for control units
        yT_nn=tf.gather(self.data['y'],yT_nn_idx,1) #now use these to retrieve y values
        yC_nn=tf.gather(self.data['y'],yC_nn_idx,1)
        y_nn=tf.dynamic_stitch([self.data['t_idx'],self.data['c_idx']],[yT_nn,yC_nn]) #stitch em back up!
        return y_nn

    def PEHEnn(self,concat_pred):
        p = self.split_pred(concat_pred)
        y_nn = self.find_ynn(p['phi']) #now its 3 plus because 
        cate_nn_err=tf.reduce_mean( tf.square( (1-2*self.data['t']) * (y_nn-self.data['y']) - (p['y1_pred']-p['y0_pred']) ) )
        return cate_nn_err

    def ATE(self,concat_pred):
        p = self.split_pred(concat_pred)
        return p['y1_pred']-p['y0_pred']


    def on_epoch_end(self, epoch, logs={}):
        concat_pred=self.model.predict(self.data['x'])
        #Calculate Empirical Metrics        
        ate_pred=tf.reduce_mean(self.ATE(concat_pred)); tf.summary.scalar('ate', data=ate_pred, step=epoch)
        pehe_nn=self.PEHEnn(concat_pred); tf.summary.scalar('cate_nn_err', data=tf.sqrt(pehe_nn), step=epoch)

        out_str=f'— cate_nn_err: {tf.sqrt(pehe_nn):.4f} '
        if self.verbose > 0: print(out_str)

## Running the Model
Now load the model, loss, and fitting boiler plate from last tutorial.
We've made three minor changes. First, we return `phi` in `concat_pred`. Second, we add `BaseMetrics` as a callback. Third, we add some code and a callback to save metrics for later viewing in Tensorboard:
```
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
```

In [7]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
 
def make_tarnet(input_dim, reg_l2):
    '''
    The first argument is the column dimension of our data.
    It needs to be specified because the functional API creates a static computational graph
    The second argument is the strength of regularization we'll apply to the output layers
    '''
    x = Input(shape=(input_dim,), name='input')
 
    # REPRESENTATION
    #in TF2/Keras it is idiomatic to instantiate a layer and pass its inputs on the same line unless the layer will be reused
    #Note that we apply no regularization to the representation layers 
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)
 
    # HYPOTHESIS
    y0_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)
 
    # second layer
    y0_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)
 
    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)
 
    #a convenience "layer" that concatenates arrays as columns in a matrix
    #this time we'll return Phi as well to calculate cate_nn_err
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions, phi])
    #the declarations above have specified the computational graph of our network, now we instantiate it
    model = Model(inputs=x, outputs=concat_pred)
 
    return model
 
# every loss function in TF2 takes 2 arguments, a vector of true values and a vector predictions
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]
 
    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]
 
    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses even though mathematically we should be using the mean
    #tf.reduce_mean and tf.reduce_sum should be equivalent, but maybe having larger error gradients makes training easier?
    return loss0 + loss1
 
### MAIN CODE ####
 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split


#make model
tarnet_model=make_tarnet(29,.01)
 
#val_split=0.2
batch_size=64
verbose=True
i = 0
tf.random.set_seed(i)
np.random.seed(i)
yt = np.concatenate([df['y'], df['t']], 1) #we'll use both y and t to compute the loss
 
# Clear any logs from previous runs
!rm -rf ./logs/fit_tarnet/
log_dir = "logs/fit_tarnet/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0),
        tensorboard_callback,
        Base_Metrics(df,verbose)
    ]



# Split data to train and validation
x_train, x_val, y_train, y_val = train_test_split(df['x'], yt, test_size=0.2, shuffle=True)


tarnet_model.compile(optimizer=Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
                    loss=regression_loss,
                    metric=regression_loss)
 
tarnet_model.fit(x=x_train,y=y_train,
                callbacks=adam_callbacks,
               # validation_split=val_split,
                epochs=20,
                batch_size=batch_size,
                validation_data=(x_val, y_val),
                verbose=verbose)
print("Done!")

2022-08-28 22:12:12.988130: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-28 22:12:12.988822: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.


Train on 24000 samples, validate on 6000 samples
Epoch 1/20
  320/24000 [..............................] - ETA: 1:14 - loss: 32.3034

2022-08-28 22:12:14.213935: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.


24000/24000 [==============================] - 21s 877us/sample - loss: 23.2318 - val_loss: 20.6877
Epoch 2/20
24000/24000 [==============================] - 19s 786us/sample - loss: 19.6667 - val_loss: 19.6233
Epoch 3/20
24000/24000 [==============================] - 19s 782us/sample - loss: 19.2149 - val_loss: 19.4029
Epoch 4/20
24000/24000 [==============================] - 19s 792us/sample - loss: 19.0502 - val_loss: 19.2549
Epoch 5/20
24000/24000 [==============================] - 19s 789us/sample - loss: 18.9203 - val_loss: 19.1469
Epoch 6/20
24000/24000 [==============================] - 19s 777us/sample - loss: 18.8041 - val_loss: 19.0312
Epoch 7/20
24000/24000 [==============================] - 19s 807us/sample - loss: 18.6873 - val_loss: 18.9079
Epoch 8/20
24000/24000 [==============================] - 19s 782us/sample - loss: 18.5796 - val_loss: 18.8355
Epoch 9/20
24000/24000 [==============================] - 20s 818us/sample - loss: 18.4725 - val_loss: 18.7065
Epoch 10/20


## Evaluating the Model Using Tensorboard

In [8]:
%tensorboard --logdir logs/fit_tarnet

# Part 2: Hyperparameter Tuning for Statistical Estimators

Now that we have metrics for model evaluation that are appropriate to causal inference, we can talk about hyperparameter optimization.
Here is a list of potentially tunable hyperparameters for TARNet:

Regularization hyperparameters:
 - $\lambda$ ($L_2$ regularization strength) for outcome layers
 - Dropout for outcome modeling layers
 - Batch normalization

Architectural hyperparameters:
  - Number of representation layers
  - Number of neurons in a representation layer
  - Number of output layers
  - Number of neurons in an output layer
  - Neuronal activation function (e.g. ELU, RELu, Sigmoid)

Optimization Hyperparameters:
 - Choice of Optimizer (e.g. SGD, ADAM)
 - Optimizer Parameters (e.g. Momentum for SGD)
 - Learning Rate Scheduling Parameters
 - Early Stopping Paremeters
 - Batch Size

We are now going to do a hyperparameter search using KerasTuner! We'll select hyperparameter settings that minimize the $\sqrt{PEHE_{nn}}$.






## Building a HyperModel using Keras Tuner

In [ ]:
# Install Keras Tuner
#!pip install keras-tuner==1.0.4

In [16]:
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
def make_hypertarnet(hp):
    """
    Neural net predictive model. The dragon has three heads.
    :param input_dim:
    :param reg:
    :return:
    """
    # hp.Choice takes hyperparam name, list of options, and default
    reg_l2=hp.Choice('l2',[.1,.01,.001],default=.01)
    input_dim=29
    inputs = Input(shape=(input_dim,), name='input')

    # representation
    rep_units = hp.Choice('rep_units', [50,100,200,400],default=200)
    phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(inputs)
    for i in range(hp.Int('rep_layers', 1, 2, default=1)):
      #pretty nifty way to dynamically add more layers!
      phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_'+str(i+2))(phi)

    # HYPOTHESIS
    hyp_units = hp.Choice('hyp_units', [20,50,100,200,400],default=100)
    y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)
    for i in range(hp.Int('hyp_layers', 1, 3, default=2)):
        y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_'+str(i+2))(y0_hidden)
        y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_'+str(i+2))(y1_hidden)
    
    # OUTPUT
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,phi])
    model = Model(inputs=inputs, outputs=concat_pred)
    
    sgd_lr = 1e-5
    momentum = 0.9
    
    optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True)
    
    model.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True),
                      loss=regression_loss,
                 metric=regression_loss)

    return model

## Tailoring Keras Tuner for our needs

Because we wish to tune models using $\sqrt{PEHE_{nn}}$ instead of the network's own loss function, we can't use the standard Keras Tuner framework. Instead, we subclass Keras Tuner and reimplement the `run_trial` method. Incidentally, this allows us to add other non-model (i.e., optimizer-related) parameters like the batch size and early-stopping patience.

 This code should look very familiar by now. The only differences is that we now have a `trial_id` for each parameter configuration which we need to use to save the model and Tensorboard logs. We also add an additional callback for saving these hyperparameter configurations in TensorBoard. Lastly the line,

`self.oracle.update_trial(trial.trial_id, {'cate_nn_err': cate_nn_err})`

reports the $\sqrt{PEHE_{nn}}$ back to Keras Tuner so that it can compare models.

In [17]:
from keras_tuner.engine import tuner_utils
from tensorboard.plugins.hparams import api as hparams_api
!rm -rf my_dir

class TarNetTuner(kt.Tuner):

    def run_trial(self, trial,dataset,*fit_args, **fit_kwargs):
        # *args and **kwargs in Python are positional (list) and keyword (dict) arguments
        verbose = fit_kwargs['verbose']

        log_dir=self.project_dir+'/trial_'+trial.trial_id
        hp = trial.hyperparameters

        batch_size = hp.Int('batch_size', 128, 256, step=64, default=128)
        stopping_patience=hp.Int('batch_size', 5, 15, step=5, default=5)

    
        hparams = tuner_utils.convert_hyperparams_to_hparams(trial.hyperparameters)
        file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
        file_writer.set_as_default()

        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        hparams_callback = hparams_api.KerasCallback(
                        writer=log_dir,
                        hparams=hparams,
                        trial_id='trial_'+trial.trial_id) 
        metrics_callback=Full_Metrics(dataset,verbose=verbose)
        callbacks = [
              TerminateOnNaN(),
              EarlyStopping(monitor='val_loss', patience=stopping_patience, min_delta=0.0001),
              ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                                min_delta=0, cooldown=0, min_lr=0),
              metrics_callback,
              tensorboard_callback,
              hparams_callback
          ]


        model = self.hypermodel.build(hp)
        model.fit(x=fit_args[0],y=fit_args[1],
                 callbacks=callbacks,
                  validation_split=fit_kwargs['validation_split'],
                  epochs=fit_kwargs['epochs'],
                  batch_size=batch_size, verbose=verbose)

        #give the metric to the hyperparameter optimization algorithm
        concat_pred=model.predict(df['x'])
        pehe_nn=metrics_callback.PEHEnn(concat_pred)
        self.oracle.update_trial(trial.trial_id, {'cate_nn_err': tf.sqrt(pehe_nn)})
        self.save_model(trial.trial_id, model)


In [20]:
tuner = TarNetTuner(
    #the oracle is the hyperoptimization algorithm
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective('cate_nn_err', 'min'),
        max_trials=10,
        seed=0    
),
        directory='my_dir',
        project_name='hypertuner',
    hypermodel=make_hypertarnet
    )
tuner.search(df, df['x'],yt, epochs=30,validation_split=.2,verbose=2)

best_trial=tuner.oracle.get_best_trials(num_trials=1)[0]
print("BEST TRIAL ID:",best_trial.trial_id)
best_model=tuner.load_model(best_trial)

Trial 4 Complete [04h 00m 13s]
cate_nn_err: 0.5859698057174683

Best cate_nn_err So Far: 0.584354817867279
Total elapsed time: 04h 21m 17s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
l2                |0.001             |0.01              
rep_units         |200               |400               
rep_layers        |1                 |2                 
hyp_units         |400               |400               
hyp_layers        |2                 |2                 
batch_size        |192               |128               

Train on 24000 samples, validate on 6000 samples
Epoch 1/30


2022-08-27 18:27:39.021276: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:499] constant folding failed: Deadline exceeded: constant folding exceeded deadline., time = 925383.438ms.
2022-08-27 18:27:39.024253: W tensorflow/core/common_runtime/process_function_library_runtime.cc:675] Ignoring multi-device function optimization failure: Deadline exceeded: meta_optimizer exceeded deadline.


2022-08-27 18:27:39.958240: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.
2022-08-27 18:27:39.959611: W tensorflow/core/framework/op_kernel.cc:1622] OP_REQUIRES failed at iterator_ops.cc:893 : Not found: Resource AnonymousIterator/AnonymousIterator393/N10tensorflow4data16IteratorResourceE does not exist.
2022-08-27 18:27:39.959899: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Not found: Resource AnonymousIterator/AnonymousIterator393/N10tensorflow4data16IteratorResourceE does not exist.
	 [[{{node IteratorGetNext}}]]


— cate_nn_err: 0.6144 


NotFoundError:  Resource AnonymousIterator/AnonymousIterator393/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext (defined at Users/kouka/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_365825]

Function call stack:
distributed_function


## Examing Hyperparameters in Tensorboard

Once KerasTuner is done, we can boot TensorBoard back up. This time we'll focus on the "HPARAMS" tab. In the "Table View" you can compare the best trial to others on the metrics we looked at before. The "Parallel Cordinates View" and "Scatter Plot Matrix View" have more information though.

Let's check out the "Parallel Cordinates View" Here you can see trends across metrics and hyperparameterizations. To the far right are the metrics we really care about: `ate_pred`,`cate_nn_err`.

In [27]:
%tensorboard --logdir my_dir/hypertuner/

# Part 3: CFRNet
Instead of using  semi-parametric corrections based on the propensity score to adjust ATE estimates, [Shalit et al., 2017](https://arxiv.org/abs/1606.03976), [Johansson et al. 2019](https://arxiv.org/abs/1903.03448), and [Johansson et al., 2020](https://arxiv.org/abs/2001.07426) take a different approach to encourage the representation function $\Phi$ to explicitly learn treatment effects: integral probability metrics. 

Integral probability metrics (IPMs) are true metrics (symmetric and obey the triangle inequality unlike KL divergence) that measure the distance between two distributions. **The key idea here is that we can add an IPM as a loss in TARNET to more explicitly encourage the representation layers to balance the covariate distribution of the treated group $T$, and the covariate distribution of the control group $C$.** The variant of TARNet with an IPM loss is called Counterfactual Regression Network (CFRNet). 
In recent years, IPMs have become very popular losses for generative adversarial networks.   

Theoretically, this group has developed generalization bounds for the $CATE$ that show that even though the counterfactual loss is unknowable, it can be bounded by the factual loss of $h$ and an IPM between the treated and control distributions. Here we use MMD as our IPM:
## Maximum Mean Discrepancy (MMD)

The  maximum  mean  discrepancy (MMD)  is  the  normed  distance between the means of two distributions $T$ and $C$,  after a kernel function has transformed them into a high-dimensional reproducing kernel Hibbert Space (RKHS). It relies on the kernel trick to calculate distances in the RKHS. The metric is built on the intuition that there is no function that would have differing expected values for $T$ and $C$ in this high-dimensional space, if $T$ and $C$ are the same distribution. Formally we can define the MMD as

$$MMD(T,C) = ||\mathbb{E}_{X \sim T}\phi(X) - \mathbb{E}_{X \sim C}\phi(X)||^2_{\mathcal{H}}$$

where $\phi$ is associated with a reproducing kernel function $k$ such as the Gaussian radial basis function kernel: $$k(X,X')=\exp({- \frac{||X-X'||^2}{2\sigma^2} }) $$

From [Johansson et al., 2020](https://arxiv.org/abs/2001.07426), an unbiased estimator for the square of the MMD from a sample of size $m$ drawn from treated distribution $T$ and a sample of size $n$ drawn from control distribution $C$ is,

$$\hat{MMD}^2_k(T,C):=\frac{1}{m(m-1)}\sum_{i=1}^m\sum_{i=j}^mk(x_i^T,x_j^T)-\frac{2}{mn}\sum_{i=1}^m\sum_{i=j}^nk(x_i^T,x_j^C)+\frac{1}{n(n-1)}\sum_{i=1}^n\sum_{i=j}^nk(x_i^C,x_j^C)$$

Architecturally CFRNet is identical to TARNet, but the loss function looks like this:

\begin{equation}
\min_{h,\Phi,IPM}\frac{1}{n}\sum_{i=1}^n \mathcal{L}(h(\Phi(x_i),t_i),y_i) + \lambda \mathcal{R}(h)+\alpha\cdot IPM(\Phi(X,|T=1),\Phi(X|T=0))\end{equation}
where $\mathcal{R}(h)$ is a model complexity term and $\lambda$ and $\alpha$ are hyperparameters. $IPM(\Phi(X,|T=1),\Phi(X|T=0))$ is an IPM distance between the covariate distributions of the treated and control distributions after they are projected into representation space.

<figure><img src=https://github.com/Gloriagao0624/Uplift/blob/main/WechatIMG61.jpeg?raw=true width="900"></figure>

## Creating a custom MMD loss

We'll begin by writing up MMD estimator described above in a custom loss object. The layer will take $\Phi$ and $T$ as inputs, and output $\hat{MMD^2}$. We'll implement the unbiased MMD estimator described above with the Guassian RBF function. Note that I've chosen to calculate losses with `tf.[reduce_mean]` here because there can be exploding gradients in weighted version of CFRNet (described below).

In [9]:
def pdist2sq(x,y):
    x2 = tf.reduce_sum(x ** 2, axis=-1, keepdims=True)
    y2 = tf.reduce_sum(y ** 2, axis=-1, keepdims=True)
    dist = x2 + tf.transpose(y2, (1, 0)) - 2. * x @ tf.transpose(y, (1, 0))
    return dist

from tensorflow.keras.losses import Loss

class CFRNet_Loss(Loss):
  #initialize instance attributes
  def __init__(self, alpha=1.,sigma=1.):
      super().__init__()
      self.alpha = alpha # balances regression loss and MMD IPM
      self.rbf_sigma=sigma #for gaussian kernel
      self.name='cfrnet_loss'
      
  def split_pred(self,concat_pred):
      #generic helper to make sure we dont make mistakes
      preds={}
      preds['y0_pred'] = concat_pred[:, 0]
      preds['y1_pred'] = concat_pred[:, 1]
      preds['phi'] = concat_pred[:, 2:]
      return preds

  def rbf_kernel(self, x, y):
    return tf.exp(-pdist2sq(x,y)/tf.square(self.rbf_sigma))

  def calc_mmdsq(self, Phi, t):
    Phic, Phit =tf.dynamic_partition(Phi,tf.cast(tf.squeeze(t),tf.int32),2)

    Kcc = self.rbf_kernel(Phic,Phic)
    Kct = self.rbf_kernel(Phic,Phit)
    Ktt = self.rbf_kernel(Phit,Phit)

    m = tf.cast(tf.shape(Phic)[0],Phi.dtype)
    n = tf.cast(tf.shape(Phit)[0],Phi.dtype)

    mmd = 1.0/(m*(m-1.0))*(tf.reduce_sum(Kcc))
    mmd = mmd + 1.0/(n*(n-1.0))*(tf.reduce_sum(Ktt))
    mmd = mmd - 2.0/(m*n)*tf.reduce_sum(Kct)
    return mmd * tf.ones_like(t)

  def mmdsq_loss(self, concat_true,concat_pred):
    t_true = concat_true[:, 1]
    p=self.split_pred(concat_pred)
    mmdsq_loss = tf.reduce_mean(self.calc_mmdsq(p['phi'],t_true))
    return mmdsq_loss

  def regression_loss(self,concat_true,concat_pred):
      y_true = concat_true[:, 0]
      t_true = concat_true[:, 1]
      p = self.split_pred(concat_pred)
      loss0 = tf.reduce_mean((1. - t_true) * tf.square(y_true - p['y0_pred']))
      loss1 = tf.reduce_mean(t_true * tf.square(y_true - p['y1_pred']))
      return loss0+loss1

  def cfr_loss(self,concat_true,concat_pred):
      lossR = self.regression_loss(concat_true,concat_pred)
      lossIPM = self.mmdsq_loss(concat_true,concat_pred)
      return lossR + self.alpha * lossIPM

      #return lossR + self.alpha * lossIPM

  #compute loss
  def call(self, concat_true, concat_pred):        
      return self.cfr_loss(concat_true,concat_pred)

## Building the model

In [10]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import Loss

def make_cfrnet(input_dim, reg_l2):

    x = Input(shape=(input_dim,), name='input')

    # representation
    phi = Dense(units=25, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=25, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=25, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=25, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=25, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=25, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,phi])
    model = Model(inputs=x, outputs=concat_pred)

    return model

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam
%load_ext tensorboard 

batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
yt = np.concatenate([df['y'], df['t']], 1)

# Clear any logs from previous runs
!rm -rf ./logs/fit_cfrnet/
log_dir = "logs/fit_cfrnet/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)




# Split data to train and validation
x_train, x_val, y_train, y_val = train_test_split(df['x'], yt, test_size=0.2, shuffle=True)


adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0),
        tensorboard_callback,
        Base_Metrics(df,verbose=verbose)
    ]


cfrnet_model=make_cfrnet(df['x'].shape[1],.01)
cfrnet_loss=CFRNet_Loss(alpha=1.0)

cfrnet_model.compile(optimizer=Adam(learning_rate=1e-5),
                      loss=cfrnet_loss,
                 metrics=[cfrnet_loss,cfrnet_loss.regression_loss,cfrnet_loss.mmdsq_loss])

cfrnet_model.fit(x=x_train,y=y_train,
                 callbacks=adam_callbacks,
                  epochs=20,
                  batch_size=batch_size,
                  validation_data=(x_val, y_val),
                  verbose=verbose)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Train on 24000 samples, validate on 6000 samples
Epoch 1/20
 2944/24000 [==>...........................] - ETA: 11s - loss: 1.4931 - cfrnet_loss: 0.4342 - regression_loss: 0.3432 - mmdsq_loss: 0.0911

2022-08-28 22:20:05.604655: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.


24000/24000 [==============================] - 6s 239us/sample - loss: 1.4470 - cfrnet_loss: 0.4012 - regression_loss: 0.3124 - mmdsq_loss: 0.0888 - val_loss: 1.4036 - val_cfrnet_loss: 0.3725 - val_regression_loss: 0.2871 - val_mmdsq_loss: 0.0854
Epoch 2/20
24000/24000 [==============================] - 3s 138us/sample - loss: 1.3725 - cfrnet_loss: 0.3556 - regression_loss: 0.2692 - mmdsq_loss: 0.0864 - val_loss: 1.3378 - val_cfrnet_loss: 0.3352 - val_regression_loss: 0.2508 - val_mmdsq_loss: 0.0843
Epoch 3/20
24000/24000 [==============================] - 3s 135us/sample - loss: 1.3119 - cfrnet_loss: 0.3231 - regression_loss: 0.2369 - mmdsq_loss: 0.0862 - val_loss: 1.2834 - val_cfrnet_loss: 0.3083 - val_regression_loss: 0.2242 - val_mmdsq_loss: 0.0842
Epoch 4/20
24000/24000 [==============================] - 3s 134us/sample - loss: 1.2614 - cfrnet_loss: 0.2998 - regression_loss: 0.2141 - mmdsq_loss: 0.0857 - val_loss: 1.2383 - val_cfrnet_loss: 0.2903 - val_regression_loss: 0.2064 - va

# Reviewing results in Tensorboard

In [12]:
%tensorboard --logdir logs/fit_cfrnet/

## Adding weights to CFRNet
A limitation of the original CFRNet is that it does not provide consistency guarantees. In [Johansson et al. 2019](https://arxiv.org/abs/1903.03448), and [Johansson et al., 2020](https://arxiv.org/abs/2001.07426), the authors introduce estimated weights $\pi(\Phi(X),T)$ derived from the propensity score to provide consistency guarantees. Interestingly, they also use these weights to relax some of the overlap assumptions as long as the weights themselves obey the positivity assumption. These weights are used to adjust both the representations when calculating the IPM and predicted outcomes.
<figure><img src=https://github.com/Gloriagao0624/Uplift/blob/main/weightedcfr.jpeg?raw=true width="900"><figcaption>Weighted CFRNet architecture introduced in Johannson et al., 2020. Purple indicates inputs, orange indicates network layers, other colors indicate output layers, and white indicates outputs. The dashes between colored shapes indicate an unspecifed number of additional hidden layers. The dashed lines on the right indicate non-gradient, plug-in computations that occur after training.</a></figcaption></figure>

Weighted CFRNet minimizes the following loss function:
\begin{equation}
\min_{h,\Phi,IPM,\pi,\lambda_h,\lambda_w}\frac{1}{n}\sum_{i=1}^n \frac{P(t_i)}{\pi(\Phi(x_i,t_i))}\cdot\mathcal{L}(h(\Phi(x_i),t_i),y_i) + \lambda_h \mathcal{R}(h)+\alpha\cdot IPM(\frac{P(1)}{\pi(\Phi(X,1))}\cdot\Phi(X,|T=1),\frac{P(0)}{\pi(\Phi(X,0))}\cdot\Phi(X|T=0))+\lambda_\pi\frac{||\pi||_2}{n}\end{equation}
where $R(h)$ is a model complexity term and $\lambda_h$, $\lambda_\pi$ and $\alpha$ are hyperparameters.

Applying the weights during the calculation of the IPM should smooth out some of the noisiness we saw from computing the IPM in tiny batches. This should reduce bias, even if weighting presents a potential variance tradeoff. The last term $\lambda_\pi\frac{||\pi||_2}{n}$ regularizes the weights to reduce variance.

In [13]:
def make_weighted_cfrnet(input_dim, reg_l2, weights_l2):

    x = Input(shape=(input_dim,), name='input')

    # representation
    phi = Dense(units=32, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=16, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=16, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=16, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)
    t_hidden = Dense(units=32, activation='elu', kernel_regularizer=regularizers.l2(weights_l2),name='t_hidden_1')(phi)

    # second layer
    #y0_hidden = Dense(units=25, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    #y1_hidden = Dense(units=25, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)
    t_hidden = Dense(units=16, activation='elu', kernel_regularizer=regularizers.l2(weights_l2),name='t_hidden_2')(t_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)
    t_predictions = Dense(units=1, activation='sigmoid', kernel_regularizer=regularizers.l2(weights_l2), name='t_predictions')(t_hidden)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,t_predictions,phi])
    model = Model(inputs=x, outputs=concat_pred)

    return model


class Weighted_CFRNet_Loss(CFRNet_Loss):
    #initialize instance attributes
    def __init__(self, prob_treat,alpha=1.0,sigma=1.0):
        super().__init__()
        self.pT=prob_treat
        self.alpha = alpha
        self.rbf_sigma=sigma
        self.name='weighted_cfrnet_loss'
    
    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['t_pred'] = (preds['t_pred'] + 0.001) / 1.002
        preds['phi'] = concat_pred[:, 3:]
        return preds
    
    #for logging purposes only
    def treatment_acc(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        #Since this isn't used as a loss, I've used tf.reduce_mean for interpretability
        return tf.reduce_mean(binary_accuracy(t_true, p['t_pred'], threshold=0.5))

    def calc_weighted_mmdsq(self, Phi, t_true, t_pred):
        t_predC, t_predT  =tf.dynamic_partition(t_pred,tf.cast(tf.squeeze(t_true),tf.int32),2) #propensity
        PhiC, PhiT =tf.dynamic_partition(Phi,tf.cast(tf.squeeze(t_true),tf.int32),2) #representation
        weightC=tf.expand_dims((1-self.pT)/(1-t_predC),axis=-1)
        weightT=tf.expand_dims(self.pT/t_predT,axis=-1)

        wPhiC = weightC * PhiC
        wPhiT = weightT * PhiT

 
        Kcc = self.rbf_kernel(wPhiC,wPhiC)
        Kct = self.rbf_kernel(wPhiC,wPhiT)
        Ktt = self.rbf_kernel(wPhiT,wPhiT)

        m = tf.cast(tf.shape(PhiC)[0],Phi.dtype)
        n = tf.cast(tf.shape(PhiT)[0],Phi.dtype)

        mmd = 1.0/(m*(m-1.0))*(tf.reduce_sum(Kcc)-m)
        mmd = mmd + 1.0/(n*(n-1.0))*(tf.reduce_sum(Ktt)-n)
        mmd = mmd - 2.0/(m*n)*tf.reduce_sum(Kct)
        return mmd * tf.ones_like(t_true)

    def weighted_mmdsq_loss(self, concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p=self.split_pred(concat_pred)
        mmdsq = tf.reduce_mean(self.calc_weighted_mmdsq(p['phi'],t_true, p['t_pred']))
        return mmdsq

    def weights(self,concat_true,concat_pred):
        p = self.split_pred(concat_pred)
        weightT=tf.expand_dims(self.pT/p['t_pred'],axis=-1)
        return tf.reduce_mean(weightT)

    def weighted_regression_loss(self, concat_true, concat_pred):
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)

        weightC=tf.expand_dims((1-self.pT)/(1.-p['t_pred']),axis=-1)
        weightT=tf.expand_dims(self.pT/p['t_pred'],axis=-1)

        loss0 = tf.reduce_mean((1. - t_true) * tf.square(y_true - p['y0_pred'])*weightC)
        loss1 = tf.reduce_mean(t_true * tf.square(y_true - p['y1_pred'])* weightT)
        return loss0 + loss1

    def call(self, concat_true, concat_pred):
        return self.weighted_regression_loss(concat_true,concat_pred) + self.alpha * self.weighted_mmdsq_loss(concat_true,concat_pred)



## Metrics callback
We just need to adjust our metric callback to account for our propensity score predictions

In [14]:
class Weighted_Metrics(Base_Metrics):
    def __init__(self,data, verbose=0):   
        super().__init__(data,verbose)

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['phi'] = concat_pred[:, 3:]

        return preds

In [17]:
batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
yt = np.concatenate([df['y'], df['t']], 1)
pT=df['t'][df['t']==1].shape[0]/df['t'].shape[0]
print("Probability of treament:", pT)
# Clear any logs from previous runs
!rm -rf ./logs/weighted_cfr 
log_dir = "logs/weighted_cfr/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)


# Split data to train and validation
x_train, x_val, y_train, y_val = train_test_split(df['x'], yt, test_size=0.2, shuffle=True)

adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0),
        tensorboard_callback,
        Weighted_Metrics(df,verbose=verbose)
    ]


weighted_cfrnet_model=make_weighted_cfrnet(df['x'].shape[1],.001,.1)
cfrnet_loss=Weighted_CFRNet_Loss(prob_treat=pT,alpha=1.)

weighted_cfrnet_model.compile(optimizer=Adam(learning_rate=1e-5),
                      loss=cfrnet_loss,
                 metrics=[cfrnet_loss,cfrnet_loss.weights,cfrnet_loss.treatment_acc,cfrnet_loss.weighted_mmdsq_loss])

weighted_cfrnet_model.fit(x=x_train,y=y_train,
                 callbacks=adam_callbacks,
                  epochs=20,
                  batch_size=batch_size,
                  validation_data=(x_val, y_val),
                  verbose=verbose)



Probability of treament: 0.6689666666666667
Train on 24000 samples, validate on 6000 samples
Epoch 1/20
 2624/24000 [==>...........................] - ETA: 18s - loss: 4.8747 - weighted_cfrnet_loss: 0.4257 - weights: 1.3376 - treatment_acc: 0.6608 - weighted_mmdsq_loss: 0.0145

2022-08-28 22:25:08.244737: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.


24000/24000 [==============================] - 10s 434us/sample - loss: 4.7783 - weighted_cfrnet_loss: 0.3903 - weights: 1.3248 - treatment_acc: 0.6938 - weighted_mmdsq_loss: 0.0155 - val_loss: 4.6738 - val_weighted_cfrnet_loss: 0.3536 - val_weights: 1.3116 - val_treatment_acc: 0.7045 - val_weighted_mmdsq_loss: 0.0152
Epoch 2/20
24000/24000 [==============================] - 6s 257us/sample - loss: 4.5932 - weighted_cfrnet_loss: 0.3394 - weights: 1.2967 - treatment_acc: 0.6985 - weighted_mmdsq_loss: 0.0159 - val_loss: 4.5034 - val_weighted_cfrnet_loss: 0.3153 - val_weights: 1.2854 - val_treatment_acc: 0.6728 - val_weighted_mmdsq_loss: 0.0165
Epoch 3/20
24000/24000 [==============================] - 7s 302us/sample - loss: 4.4295 - weighted_cfrnet_loss: 0.3059 - weights: 1.2721 - treatment_acc: 0.6764 - weighted_mmdsq_loss: 0.0174 - val_loss: 4.3481 - val_weighted_cfrnet_loss: 0.2885 - val_weights: 1.2621 - val_treatment_acc: 0.6608 - val_weighted_mmdsq_loss: 0.0179
Epoch 4/20
24000/240

Epoch 17/20
24000/24000 [==============================] - 9s 389us/sample - loss: 2.8046 - weighted_cfrnet_loss: 0.2002 - weights: 1.0737 - treatment_acc: 0.6714 - weighted_mmdsq_loss: 0.0086 - val_loss: 2.7585 - val_weighted_cfrnet_loss: 0.1997 - val_weights: 1.0700 - val_treatment_acc: 0.6583 - val_weighted_mmdsq_loss: 0.0089
Epoch 18/20
24000/24000 [==============================] - 9s 383us/sample - loss: 2.7128 - weighted_cfrnet_loss: 0.1987 - weights: 1.0662 - treatment_acc: 0.6714 - weighted_mmdsq_loss: 0.0082 - val_loss: 2.6681 - val_weighted_cfrnet_loss: 0.1986 - val_weights: 1.0627 - val_treatment_acc: 0.6583 - val_weighted_mmdsq_loss: 0.0083
Epoch 19/20
24000/24000 [==============================] - 7s 291us/sample - loss: 2.6234 - weighted_cfrnet_loss: 0.1976 - weights: 1.0593 - treatment_acc: 0.6715 - weighted_mmdsq_loss: 0.0078 - val_loss: 2.5799 - val_weighted_cfrnet_loss: 0.1976 - val_weights: 1.0561 - val_treatment_acc: 0.6583 - val_weighted_mmdsq_loss: 0.0080
Epoch 2

## Reviewing results in Tensorboard

The most noticeable thing in our logs is that the `mmdsq_loss` is significantly smoother than in the previous trial. FWIW, the authors also demonstrate in the paper that including the weights lessens the dependence on $\alpha$ because the weights are learned adaptively.

In [18]:
%tensorboard --logdir logs/weighted_cfr

# Part 4: Drangonnet: semi-parametric extensions to TARNet 

 
Beyond outcome modeling, another approach to reducing confounding is adjusting for selection into treatment. This is typically done using the *propensity score*. If the $ATE$ is identifiable by adjusting for $X$, then the propensity score $\pi(X,T)=P(T|X)$ is sufficient to identify the $ATE$ as well (Rosenbaum and Rubin, 1983). We can estimate the ATE using inverse propensity score weighting:
 
$\hat{ATE}=[\frac{T}{\pi(X,T)}-\frac{1-T}{\pi(X,1-T)}]\cdot Y$
 
To use the IPW estimator with a neural network, we can trivially add a third "head" to predict the treatment from the representation $\Phi$ (actually if we *just* wanted to do IPW we don't need the other two heads at all),
 
$\hat{ATE}=[\frac{T}{\pi(\Phi(X),T)}-\frac{1-T}{\pi(\Phi(X),1-T)}]\cdot Y$
 
 This is the "Dragonnet" architecture from [Shi et al., 2019](https://arxiv.org/pdf/1906.02120.pdf).

<figure><img src=https://github.com/Gloriagao0624/Uplift/blob/main/drangonet.jpeg?raw=true width="900"><figcaption>Dragonnet architecture introduced in Shi et al., 2019. This is just TARNet with a third head (single neuron) predicting the propensity score $P(T)=\pi(\Phi(X),T)$. Purple represents input data, orange represents representation layers. Red, blue, and green are output layers for control outcome, treated outcome, and propensity score, repsectively. The "nudge" parameter is in yellow. Dashes between arrows indicate possible additional hidden layers. Non-gradient, plug-in computation of $\hat{CATE}$, indicated by dashed lines between white shapes, occurs after training. Figure taken from accompanying review paper.</a></figcaption></figure>
 
The third head could be implemented as a single neuron (as in DragonNet) or using additional layers as in ([Johansson et al. 2018](https://arxiv.org/abs/1903.03448), and [Johansson et al., 2020](https://arxiv.org/abs/2001.07426)) to produce a scalar propensity score $P(T|\Phi(X))=\pi(\Phi(X),T)$.
 
The loss function for this network looks like this:
$$\underset{\phi,\pi,h}{\arg \min}\ MSE(Y,h(\Phi(X),T)) + \alpha \cdot \text{BCE}(T,\pi(\Phi(X),T))$$
with $\alpha$ being a hyperparameter to balance the two objectives.
 
Below we break down more sophisticated ways that the propensity score is used in [Shi et al., 2019](https://arxiv.org/pdf/1906.02120.pdf) from semi-parametric estimation theory.

## Semi-parametric theory in three paragraphs

The application of semi-parametric theory to causal inference (as far as I understand it), is focused on estimating a target parameter of a distribution $P$ of treatment effects $T(P):=ATE$. While we do not know the true distribution of treatment effects because we lack counterfactuals, we do know some parameters generating this distribution (e.g., the treatment assignment mechanism). We can encode these  constraints in the form of a likelihood that parametrically defines a set of possible approximate distributions of $P$ from our existing data that we'll call $\mathcal{P}$. Within this set there is a sample-inferred distribution $\tilde{P}\in\mathcal{P}$, that we can use to estimate $T(P)$ using $T(\tilde{P})$.

### Picking $\tilde{P}$

Regardless of $\tilde{P}$ chosen, $\tilde{P}\neq P \therefore T(\tilde{P})\neq T(P)$. We don't really know how to pick $\tilde{P}$ with finite data to get the best estimate $T(\tilde{P})$. We can maximize our likelihood function to pick $T(\tilde{P})$, but there are a lot of "nuisance" parameters in the likelihood that are not our target that we don't really care about estimating accurately, so this won't necessarily give us the best estimate of $T(P)$. This is where **influence curves** come in. 
 
 We're going to define a "nudge" parameter $\epsilon$ that moves $\tilde{P}$ closer to $P$ (thus moving $T(\tilde{P})$ closer to $T(P)$). An influence curve of $T(P)$ tells us how changes in $\epsilon$ will induce changes in $T(P+\epsilon(\tilde{P}-P))$. We'll use this influence curve to fit $\epsilon$ to get the best approximation of $T(P)$ that we can. In particular, there is a specific **efficient influence curve (EIC)** that provides us with the lowest variance (efficient) estimates of $T(P)$.



## AIPW

The augmented inverse propensity weighting estimator (AIPW or sometimes AIPTW) is an estimator
that solves the efficient influence curve estimating equation for the ATE directly (i.e.,  without a nudge parameter). 

In AIPW (and TMLE), we set the mean of the EIC estimating equation equal to zero which allows us to use it to estimate the $ATE$ linearly. The estimating equation models both the outcome and the treatment. We can specify it as:

$EIC = \frac{1}{N}\sum_{i=1}^N{[(\frac{T}{\pi(\Phi(X),1)}-\frac{1-T}{\pi(\Phi(X),0)})[Y-h(\Phi(X),T)] +[h(\Phi(X),1)-h(\Phi(X),0)]}]-ATE$

$(\text{Set mean of EIC to 0})$


$ATE = \frac{1}{N}\sum_{i=1}^N{[\underbrace{\underbrace{(\frac{T}{\pi(\Phi(X),1)}-\frac{1-T}{\pi(\Phi(X),0)})}_{\text{Treatment Modeling}}\times\underbrace{[Y-h(\Phi(X),T)]}_{\text{Residual Confounding}}}_{\text{Adjustment}} +\underbrace{[h(\Phi(X),1)-h(\Phi(X),0)]}_{\text{Outcome Modeling}}}]$

There is another interpretation of the AIPW as a "doubly robust" estimator. As a doubly robust estimator, we are effectively using Dragonnet to do outcome modeling of $T(\tilde{P})$ in the second term, but account for any residual confounding (second part of the first term) using a function of the propensity score. Doubly robust estimators are appealing because they will produce a consistent estimate of the $ATE$ if either $\pi$ or $h$ is estimated consistently, and are efficient if both are estimated correctly to solve the estimating equation.

In [19]:
#!pip install -q tensorflow==2.8.0
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import Loss
def make_aipw(input_dim, reg_l2):

    x = Input(shape=(input_dim,), name='input')
    # representation
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #propensity prediction
    #Note that the activation is actually sigmoid, but we will squish it in the loss function for numerical stability reasons
    t_prediction = Dense(units=1,activation=None,name='t_prediction')(phi)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,t_prediction,phi])
    model = Model(inputs=x, outputs=concat_pred)
    return model

class Base_Loss(Loss):
    #initialize instance attributes
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha
        self.name='standard_loss'

    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['phi'] = concat_pred[:, 3:]
        return preds

    #for logging purposes only
    def treatment_acc(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        #Since this isn't used as a loss, I've used tf.reduce_mean for interpretability
        return tf.reduce_mean(binary_accuracy(t_true, tf.math.sigmoid(p['t_pred']), threshold=0.5))

    def treatment_bce(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        lossP = tf.reduce_sum(binary_crossentropy(t_true,p['t_pred'],from_logits=True))
        return lossP
    
    def regression_loss(self,concat_true,concat_pred):
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - p['y0_pred']))
        loss1 = tf.reduce_sum(t_true * tf.square(y_true - p['y1_pred']))
        return loss0+loss1

    def standard_loss(self,concat_true,concat_pred):
        lossR = self.regression_loss(concat_true,concat_pred)
        lossP = self.treatment_bce(concat_true,concat_pred)
        return lossR + self.alpha * lossP

    #compute loss
    def call(self, concat_true, concat_pred):        
        return self.standard_loss(concat_true,concat_pred)
        



Now let's add AIPW to our callback.

In [20]:
from tensorflow.keras.callbacks import Callback

'''
def pdist2sq(A, B):
    #helper for PEHEnn
    #calculates squared euclidean distance between rows of two matrices  
    #https://gist.github.com/mbsariyildiz/34cdc26afb630e8cae079048eef91865
    # squared norms of each row in A and B
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)    
    # na as a row and nb as a column vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])
    # return pairwise euclidean difference matrix
    D = tf.sqrt(tf.maximum(na - 2*tf.matmul(A, B, False, True) + nb, 0.0))
    return D
'''

def pdist2sq(x,y):
    x2 = tf.reduce_sum(x ** 2, axis=-1, keepdims=True)
    y2 = tf.reduce_sum(y ** 2, axis=-1, keepdims=True)
    dist = x2 + tf.transpose(y2, (1, 0)) - 2. * x @ tf.transpose(y, (1, 0))
    return dist

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class AIPW_Metrics(Callback):
    def __init__(self,data, verbose=0):   
        super(AIPW_Metrics, self).__init__()
        self.data=data #feed the callback the full dataset
        self.verbose=verbose

        #needed for PEHEnn; Called in self.find_ynn
        self.data['o_idx']=tf.range(self.data['t'].shape[0])
        self.data['c_idx']=self.data['o_idx'][self.data['t'].squeeze()==0] #These are the indices of the control units
        self.data['t_idx']=self.data['o_idx'][self.data['t'].squeeze()==1] #These are the indices of the treated units
    
    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = concat_pred[:, 0].reshape(-1, 1)
        preds['y1_pred'] = concat_pred[:, 1].reshape(-1, 1)
        preds['t_pred'] = concat_pred[:, 2]
        preds['phi'] = concat_pred[:, 3:]
        return preds

    def find_ynn(self, Phi):
        #helper for PEHEnn
        PhiC, PhiT =tf.dynamic_partition(Phi,tf.cast(tf.squeeze(self.data['t']),tf.int32),2) #separate control and treated reps
        dists=tf.sqrt(pdist2sq(PhiC,PhiT)) #calculate squared distance then sqrt to get euclidean
        yT_nn_idx=tf.gather(self.data['c_idx'],tf.argmin(dists,axis=0),1) #get c_idxs of smallest distances for treated units
        yC_nn_idx=tf.gather(self.data['t_idx'],tf.argmin(dists,axis=1),1) #get t_idxs of smallest distances for control units
        yT_nn=tf.gather(self.data['y'],yT_nn_idx,1) #now use these to retrieve y values
        yC_nn=tf.gather(self.data['y'],yC_nn_idx,1)
        y_nn=tf.dynamic_stitch([self.data['t_idx'],self.data['c_idx']],[yT_nn,yC_nn]) #stitch em back up!
        return y_nn

    def PEHEnn(self,concat_pred):
        p = self.split_pred(concat_pred)
        y_nn = self.find_ynn(p['phi']) #now its 3 plus because 
        cate_nn_err=tf.reduce_mean( tf.square( (1-2*self.data['t']) * (y_nn-self.data['y']) - (p['y1_pred']-p['y0_pred']) ) )
        return cate_nn_err

    def ATE(self,concat_pred):
        p = self.split_pred(concat_pred)
        return p['y1_pred']-p['y0_pred']

   
    #THIS IS THE NEW PART
    def AIPW(self,concat_pred):
        p = self.split_pred(concat_pred)
        t_pred=tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        y_pred = p['y0_pred'] * (1 - self.data['t']) + p['y1_pred'] * self.data['t']
        #cc stands for clever covariate which is I think what it's called in TMLE lit
        cc = self.data['t'] * (1.0 / p['t_pred']) - (1.0 - self.data['t']) / (1.0 - p['t_pred'])
        cate = cc * (self.data['y'] - y_pred) + p['y1_pred'] - p['y0_pred']
        return cate

    def on_epoch_end(self, epoch, logs={}):
        concat_pred=self.model.predict(self.data['x'])
        #Calculate Empirical Metrics        
        ate_pred=tf.reduce_mean(self.ATE(concat_pred)); tf.summary.scalar('ate', data=ate_pred, step=epoch)
        pehe_nn=self.PEHEnn(concat_pred); tf.summary.scalar('cate_nn_err', data=tf.sqrt(pehe_nn), step=epoch)
        aipw=tf.reduce_mean(self.AIPW(concat_pred)); tf.summary.scalar('aipw', data=aipw, step=epoch)
        out_str=f' — cate_nn_err: {tf.sqrt(pehe_nn):.4f} '
        
        if self.verbose > 0: print(out_str)

In [21]:
import datetime
%load_ext tensorboard

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam

batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
yt = np.concatenate([df['y'], df['t']], 1)

# Clear any logs from previous runs
!rm -rf ./logs/fit_dragonnet/
log_dir = "logs/fit_dragonnet/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# Split data to train and validation
x_train, x_val, y_train, y_val = train_test_split(df['x'], yt, test_size=0.2, shuffle=True)

adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0),
        tensorboard_callback,
        AIPW_Metrics(df,verbose=verbose)
    ]  
    
    
    

aipw_model=make_aipw(29,.01)
aipw_loss=Base_Loss(alpha=1.0)

aipw_model.compile(optimizer=Adam(lr=1e-5),
                    loss=aipw_loss,
                    metrics=[aipw_loss,aipw_loss.regression_loss,aipw_loss.treatment_acc]
                   )

aipw_model.fit(x=x_train,y=y_train,
                  callbacks=adam_callbacks,
                  validation_data=(x_val, y_val),
                  epochs=20,
                  batch_size=batch_size,
                  verbose=verbose)




The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Train on 24000 samples, validate on 6000 samples
Epoch 1/20
  384/24000 [..............................] - ETA: 2:01 - loss: 32.3371 - standard_loss: 24.3027 - regression_loss: 23.6070 - treatment_acc: 0.5130

2022-08-28 23:15:07.127895: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.


24000/24000 [==============================] - 41s 2ms/sample - loss: 23.6108 - standard_loss: 15.6069 - regression_loss: 14.9181 - treatment_acc: 0.6092 - val_loss: 21.2507 - val_standard_loss: 13.2721 - val_regression_loss: 12.5899 - val_treatment_acc: 0.6599
Epoch 2/20
24000/24000 [==============================] - 33s 1ms/sample - loss: 20.2715 - standard_loss: 12.3366 - regression_loss: 11.6594 - treatment_acc: 0.6753 - val_loss: 20.3128 - val_standard_loss: 12.4078 - val_regression_loss: 11.7358 - val_treatment_acc: 0.6823
Epoch 3/20
24000/24000 [==============================] - 33s 1ms/sample - loss: 19.8771 - standard_loss: 12.0155 - regression_loss: 11.3481 - treatment_acc: 0.6961 - val_loss: 20.0673 - val_standard_loss: 12.2381 - val_regression_loss: 11.5753 - val_treatment_acc: 0.6955
Epoch 4/20
24000/24000 [==============================] - 48s 2ms/sample - loss: 19.7227 - standard_loss: 11.9358 - regression_loss: 11.2777 - treatment_acc: 0.7111 - val_loss: 19.9138 - val_s

Epoch 20/20
24000/24000 [==============================] - 33s 1ms/sample - loss: 18.3196 - standard_loss: 11.6638 - regression_loss: 11.0950 - treatment_acc: 0.7497 - val_loss: 18.5873 - val_standard_loss: 11.9579 - val_regression_loss: 11.3850 - val_treatment_acc: 0.7355


## Reviewing results in Tensorboard

Let's do a quick comparison to last tutorial where we ran the exact same network without the propensity score loss.

If we look at `treatment_acc`, it's clear that Dragonnet is learning the treatment information. We can also see that there is only a very slight penalty in the network's ability to predict the outcomes (`ate_err`).

It's hard to say whether any other performance differences are significant without doing hyperparameter tuning under both scenarios and looking across multiple simulations. For what it's worth, the `aipw_err` is slightly worse than the raw `ate_err` but it's pretty close, and the statistical guarantees are definitely worth something.  

In [22]:
%tensorboard --logdir logs/fit_dragonnet/

## Implementing Targeted Regularization
 We'll need to start by adding $\epsilon$ as a parameter in our neural network. Shi does this by creating a [custom layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) she calls `EpsilonLayer`. 

In [23]:
from tensorflow.keras.layers import Layer
class EpsilonLayer(Layer):

    def __init__(self):
        super(EpsilonLayer, self).__init__()

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.epsilon = self.add_weight(name='epsilon',
                                       shape=[1, 1],
                                       initializer='RandomNormal',
                                       #  initializer='ones',
                                       trainable=True)
        super(EpsilonLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, **kwargs):
        #note there is only one epsilon were just duplicating it for conformability
        return self.epsilon * tf.ones_like(inputs)[:, 0:1]

def make_dragonnet(input_dim, reg_l2):

    x = Input(shape=(input_dim,), name='input')
    # representation
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=200, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #propensity prediction
    #Note that the activation is actually sigmoid, but we will squish it in the loss function for numerical stability reasons
    t_predictions = Dense(units=1,activation=None,name='t_prediction')(phi)
    #Although the epsilon layer takes an input, it really just houses a free parameter. 
    epsilons = EpsilonLayer()(t_predictions)
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,t_predictions,epsilons,phi])
    model = Model(inputs=x, outputs=concat_pred)
    return model



In [24]:
class TarReg_Loss(Base_Loss):
    #initialize instance attributes
    def __init__(self, alpha=1,beta=1):
        super().__init__()
        self.alpha = alpha
        self.beta=beta
        self.name='tarreg_loss'

    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3] #we're moving epsilon into slot three
        preds['phi'] = concat_pred[:, 4:]
        return preds

    def calc_hstar(self,concat_true,concat_pred):
        #step 2 above
        p=self.split_pred(concat_pred)
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]

        t_pred = tf.math.sigmoid(concat_pred[:, 2])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        y_pred = t_true * p['y1_pred'] + (1 - t_true) * p['y0_pred']

        #calling it cc for "clever covariate" as in SuperLearner TMLE literature
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + p['epsilon'] * cc
        return h_star

    def call(self,concat_true,concat_pred):
        y_true = concat_true[:, 0]

        standard_loss=self.standard_loss(concat_true,concat_pred)
        h_star=self.calc_hstar(concat_true,concat_pred)
        #step 3 above
        targeted_regularization = tf.reduce_sum(tf.square(y_true - h_star))

        # final
        loss = standard_loss + self.beta * targeted_regularization
        return loss

Now we update our callback so that it computes $h*$ and the final, plug-in $\hat{ATE}_{\text{TR}}$. Looking at the Latex may again be helpful. We can save some code lines by subclassing.



In [26]:
class TarReg_Metrics(AIPW_Metrics):
    def __init__(self,data, verbose=0):   
        super().__init__(data,verbose)

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = concat_pred[:, 0].reshape(-1, 1)
        preds['y1_pred'] = concat_pred[:, 1].reshape(-1, 1)
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3]
        preds['phi'] = concat_pred[:, 4:]
        return preds
    
    def compute_hstar(self,y0_pred,y1_pred,t_pred,t_true,epsilons):
        #helper for calculating the targeted regularization cate
        y_pred = t_true * y1_pred + (1 - t_true) * y0_pred
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + epsilons * cc
        return h_star
    
    def TARREG_CATE(self,concat_pred):
        #Final calculation of Targeted Regularization loss
        p = self.split_pred(concat_pred)
        t_pred = tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi       
        hstar_0=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.zeros_like(p['epsilon']),p['epsilon'])
        hstar_1=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.ones_like(p['epsilon']),p['epsilon'])
        return hstar_1-hstar_0

    def on_epoch_end(self, epoch, logs={}):
        concat_pred=self.model.predict(self.data['x'])
        #Calculate Empirical Metrics        
        aipw_pred=tf.reduce_mean(self.AIPW(concat_pred)); tf.summary.scalar('aipw', data=aipw_pred, step=epoch)
        ate_pred=tf.reduce_mean(self.ATE(concat_pred)); tf.summary.scalar('ate', data=ate_pred, step=epoch)
        tarreg_pred=tf.reduce_mean(self.TARREG_CATE(concat_pred)); tf.summary.scalar('tarreg_pred', data=tarreg_pred, step=epoch)
        pehe_nn=self.PEHEnn(concat_pred); tf.summary.scalar('cate_nn_err', data=tf.sqrt(pehe_nn), step=epoch)
        
        out_str=f'— cate_nn_err: {tf.sqrt(pehe_nn):.4f}'
        
        if self.verbose > 0: print(out_str)

In [28]:
import tensorflow as tf
import numpy as np
import datetime
%load_ext tensorboard

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam

batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
yt = np.concatenate([df['y'], df['t']], 1)

# Clear any logs from previous runs
!rm -rf ./logs/dragonnet_tarReg/
log_dir = "logs/dragonnet_tarReg/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

# Split data to train and validation
x_train, x_val, y_train, y_val = train_test_split(df['x'], yt, test_size=0.2, shuffle=True)

adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0),
        tensorboard_callback,
        TarReg_Metrics(df,verbose=verbose)
    ]  



dragonnet_model=make_dragonnet(df['x'].shape[1],.01)
tarreg_loss=TarReg_Loss(alpha=1)

dragonnet_model.compile(#optimizer=SGD(learning_rate=sgd_lr, momentum=momentum, nesterov=True),
                 optimizer=Adam(lr=1e-5),
                 loss=tarreg_loss,
                 metric=[tarreg_loss,tarreg_loss.regression_loss,tarreg_loss.treatment_acc])

dragonnet_model.fit(x=x_train,y=y_train,
                 callbacks=adam_callbacks,
                 validation_data=(x_val, y_val),
                  epochs=20,
                  batch_size=batch_size,
                  verbose=verbose)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Train on 24000 samples, validate on 6000 samples
Epoch 1/20
  448/24000 [..............................] - ETA: 1:02 - loss: 65.7172

2022-08-28 23:58:17.899914: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.


24000/24000 [==============================] - 63s 3ms/sample - loss: 57.9892 - val_loss: 36.8352
Epoch 2/20
24000/24000 [==============================] - 67s 3ms/sample - loss: 35.0790 - val_loss: 33.9772
Epoch 3/20
24000/24000 [==============================] - 69s 3ms/sample - loss: 33.2327 - val_loss: 33.4493
Epoch 4/20
24000/24000 [==============================] - 68s 3ms/sample - loss: 32.8469 - val_loss: 33.2100
Epoch 5/20
24000/24000 [==============================] - 65s 3ms/sample - loss: 32.6399 - val_loss: 33.0752
Epoch 6/20
24000/24000 [==============================] - 71s 3ms/sample - loss: 32.4803 - val_loss: 32.9288
Epoch 7/20
24000/24000 [==============================] - 66s 3ms/sample - loss: 32.3478 - val_loss: 32.8039
Epoch 8/20
24000/24000 [==============================] - 68s 3ms/sample - loss: 32.2247 - val_loss: 32.7543
Epoch 9/20
24000/24000 [==============================] - 71s 3ms/sample - loss: 32.1191 - val_loss: 32.5973
Epoch 10/20
24000/24000 [=====

## Reviewing results in Tensorboard


In [29]:
%tensorboard --logdir logs/dragonnet_tarReg/

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 39708.